In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
from django.db import models
# Create your models here.
import psycopg2
#from config import config
from django.test.html import Element
import os

from configparser import ConfigParser


def config(section='postgresql'):
    if os.getcwd()[0:12]!="/home/traore":
    	filename=os.getcwd()+'/database.ini'
    else:
    	filename=os.getcwd()+'/database2.ini'
    # create a parser
    parser = ConfigParser()
    # read config file
    parser.read(filename)
    # get section, default to postgresql
    db = {}
    if parser.has_section(section):
        params = parser.items(section)
        for param in params:
            db[param[0]] = param[1]
    else:
        raise Exception('Section {0} not found in the {1} file'.format(section, filename))
 
    return db




def connect():
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        # read connection parameters
        params = config()
        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**params)
        # create a cursor
        cur = conn.cursor()
        # execute a statement
        print('PostgreSQL database version:')
        cur.execute('SELECT version()')
        # display the PostgreSQL database server version
        db_version = cur.fetchone()
        print(db_version)
     # close the communication with the PostgreSQL
        cur.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()
            print('Database connection closed.')


def read_data_sql(requete,element):
    """
    input requete outupt data
    requete='select * from chips.experiment;'
    """
    conn = None
    params = config()
    conn = psycopg2.connect(**params)
    cursor = conn.cursor()#cursor_factory=psycopg2.extras.DictCursor
    cursor.execute(requete)
    colnames = [desc[0] for desc in cursor.description]
    memory = cursor.fetchall()
    conn.close()
    cpt=0
    data=dict()
    for ele in range(len(memory)):
        cpt=cpt+1
        col=[]
        for ele1 in memory[ele]:
            col.append(str(ele1))
        dictionary ={'_key':cpt,"_value":dict(zip(colnames,col))}
        data[cpt]=dictionary
    name=element+'.csv'     
    return data

def lecture_query_tage(file_query):
    """
    function sert a lire les requetes prédefinir seulement
    """
    query_tag=dict()
    with open(file_query,"r") as fiche:
        data=fiche.readlines()
        for line in data:
            if line[0]!="#":
                if line[0:9]=="Tag_query":
                    _key=line[11:].replace('\n','').replace('"','')
                elif line[0:5]=="query":
                    query_tag[_key]=(line[6:].replace('\n','').replace('"',''))            
    return query_tag


repertoire_requete=lecture_query_tage(os.getcwd()+'/query_tag.ini')

def execution_requete(element,value_send,colname_send,order_ele=None):
    ### parametres
    if os.getcwd()[0:12]!="/home/traore":
    	my_ppty="chips"
    else:
    	my_ppty="public" #CHIPS

   
    # creation/tri des infos :(creation de structures de données)
    ###Todo
    
    # Bilan Chiffres nbr de puces par especes
    # pas Arabidopsis dans nouveau champ common_name d'array_type et 
    # analysis_type='Arrays'
    # partie requete
    my_querySpecies="select array_type_name, count(hybridization_id) "+\
    "from "+my_ppty+".hybridization "+\
    "where project_id in (select project_id from "+my_ppty+".project where is_public='yes') "+\
    "and experiment_id in (select experiment_id from "+my_ppty+".experiment "+\
    "where analysis_type='Arrays') "+\
    "and array_type_name in "+\
    "(select a.array_type_name from "+my_ppty+".array_type a where a.common_name<>'Arabidopsis')"+\
    "group by array_type_name order by array_type_name;"
    
    #Pour le nombre de ref biblio
    my_reqstat ="Select count(*) from "+my_ppty+".biblio_list;"
    
    
    #my_req_special="select o.organism_name,count(distinct ss.project_id) from "+my_ppty+".sample_source ss,"+my_ppty+".organism o where ss.project_id in( select project_id from "+my_ppty+".project where is_public='yes')  group by o.organism_name;"
    my_req_special="select o.organism_name,count(distinct ss.project_id) from "+my_ppty+".sample_source ss,"+my_ppty+".organism o where ss.project_id in( select project_id from "+my_ppty+".project ) and ss.organism_id=o.organism_id group by o.organism_name;" #where is_public='yes'
    
    my_req_recherche="SELECT * FROM global_search_element('Arabidopsis',param_schemas:=array['chips']);"
    
    my_req_recherche1="select project_id,Experiment_type,Experiment_name from "+my_ppty+".experiment where project_id in (select project_id from "+my_ppty+".project where is_public='yes');"
    
    #experiement="select * from chips.experiment where project_id in (select project_id from chips.project where is_public='yes'));"
    
    experiement="select * from (select S.project_name,O.project_id,O.experiment_name,O.experiment_type from "+my_ppty+".experiment O,"+my_ppty+".project S where O.project_id in (select project_id from "+my_ppty+".project where is_public='yes')) df limit 100;"
    
    
    try:
        my_req_recherche_av="select * from "+colname_send+" where (is_public='yes' and project_name like '%"+value_send+"%');"
    except:
        pass
    if element=='my_reqinfo':
        requete=my_reqinfo
    elif element=='my_querySpecies':
        requete=my_querySpecies
    elif element=='my_reqstat':
        requete=my_reqstat    
    elif element=='my_req_special':
        requete=my_req_special    
    elif element=='my_req_recherche':
        requete=my_req_recherche        
    elif element=='my_req_recherche_av' and colname_send!=None and value_send!=None:
        requete=my_req_recherche_av     
    elif element=='my_req_recherche1':
        requete=my_req_recherche1
    elif element=='experiement':
        requete=experiement  
    elif (element=='get_project' or element=='get_contact' or element=='my_experience' or 'my_treatment' or 'list_project' or 'type_analysis' or 'type_arrays' or 'all_experiment'):
        requete=repertoire_requete[element].replace('schema.',my_ppty+'.')
        if value_send!=None:
            requete=requete.replace('value_send',value_send)
        else:
            requete=requete
        print(requete)
    else:
        try:
            requete=eval(element)
        except:
            requete=None
            pass
    
    my_req_special="select o.organism_name,count(distinct ss.project_id) from "+my_ppty+".sample_source ss,"+my_ppty+".organism o where ss.project_id in( select project_id from "+my_ppty+".project where is_public='yes') and ss.organism_id=o.organism_id group by o.organism_name;"
    
    
    
    if  element=='my_req_special' and order_ele==None:
         requete=my_req_special  
    elif element=='my_req_special' and order_ele!=None:    
        requete="select * from ("+requete.replace(";",')')+' dfg order by dfg.'+order_ele+";"
    elif  element=='my_querySpecies' and order_ele==None:
         requete=my_querySpecies  
    elif element=='my_querySpecies' and order_ele!=None:    
        requete="select * from ("+requete.replace(";",')')+' dfg order by dfg.'+order_ele+";"
    elif  element=='my_req_recherche1' and order_ele==None:
         requete=my_req_recherche1  
    elif element=='my_req_recherche1' and order_ele!=None:    
        requete="select * from ("+requete.replace(";",')')+' dfg order by dfg.'+order_ele+";"
    if requete!=None:
        return read_data_sql(requete,element)    
    else:
        print("error") 

In [2]:
data=execution_requete('select_all_tab','','')

SELECT project.project_id, sample.organ,sample.sample_id,sample.sample_name, experiment.experiment_name,experiment.experiment_id,experiment.experiment_type, experiment.array_type, project.project_name, project.title,organism.organism_id,organism.organism_name FROM chips.organism,chips.sample_source,chips.experiment, chips.project, chips.sample WHERE project.project_id = experiment.project_id AND   sample.project_id = experiment.project_id AND experiment.experiment_id=sample.experiment_id and sample_source.experiment_id=sample.experiment_id and sample.project_id = sample_source.project_id and sample_source.organism_id = organism.organism_id limit 10;


In [25]:
data

{1: {'_key': 1,
  '_value': {'array_type': 'CAGE',
   'experiment_id': '120',
   'experiment_name': 'seedlings_10j_Columbia_WT_vs_ddm1',
   'experiment_type': 'gene knock out',
   'organ': 'whole plant',
   'organism_id': '1',
   'organism_name': 'Arabidopsis thaliana',
   'project_id': '81',
   'project_name': 'CAGE-06_02_ddm1-VIBchips_PRIVATE',
   'sample_id': '1071',
   'sample_name': 'Col_ddm1_10j',
   'title': 'Microarray analysis of transcriptionnal changes induced  by the ddm1 mutation'}},
 2: {'_key': 2,
  '_value': {'array_type': 'CAGE',
   'experiment_id': '120',
   'experiment_name': 'seedlings_10j_Columbia_WT_vs_ddm1',
   'experiment_type': 'gene knock out',
   'organ': 'whole plant',
   'organism_id': '1',
   'organism_name': 'Arabidopsis thaliana',
   'project_id': '81',
   'project_name': 'CAGE-06_02_ddm1-VIBchips_PRIVATE',
   'sample_id': '1070',
   'sample_name': 'Col_WT_10j',
   'title': 'Microarray analysis of transcriptionnal changes induced  by the ddm1 mutation'}}

In [4]:
sepcol='#'
sep=':'
diction={}
dictionlev2={}
dictionlev3={}
for key in data.keys():
    level1='project_id:'+data[key]['_value']['project_id']+sepcol+'project_name:'+data[key]['_value']['project_name']
    level2='experiment_id:'+data[key]['_value']['experiment_id']+sepcol+'experiment_name:'+data[key]['_value']['experiment_name']+sepcol+'experiment_type:'+data[key]['_value']['experiment_type']+sepcol+'array_type:'+data[key]['_value']['array_type']
    level3='organ:'+data[key]['_value']['organ']#'sample_id:'+data[key]['_value']['sample_id']+'sample_name:'+data[key]['_value']['sample_name']+
    level4='organism_name:'+data[key]['_value']['organism_name']+sepcol+'organism_id:'+data[key]['_value']['organism_id']
    if level1 not in diction.keys():
        diction[level1]=[level2]
    else:
        dat=diction[level1]
        dat.append(level2)
        diction[level1]=list(set(dat))
    if level1+sepcol+level2 not in dictionlev2.keys():
        dictionlev2[level1+sepcol+level2]=[level3+sepcol+level4]
    else:
        dat2=dictionlev2[level1+sepcol+level2]
        dat2.append(level3+sepcol+level4)
        dictionlev2[level1+sepcol+level2]=list(set(dat2))

In [5]:
#dictionlev2
text=''
cpt=0
for key in diction.keys():
    cpt=cpt+1
    text=text+"{'_key':"+str(cpt)+','
    for element in key.split('#'):
        text=text+"'"+element.split(':')[0]+"':"+"'"+element.split(':')[1]+"',"
    cpt2=0
    for el in diction[key]:
        cpt2=cpt2+1
        text=text+"'experiment"+str(cpt2)+"':{"
        for val in el.split('#'):
            text=text+"'"+val.split(':')[0]+"':'"+val.split(':')[1]+"'"+','
        
        cpt3=0
        for el2 in dictionlev2[key+sepcol+el]:
            cpt3=cpt3+1
            text=text+"'organism"+str(cpt3)+"':{"
            for val2 in el2.split('#'):
                text=text+"'"+val2.split(':')[0]+"':'"+val2.split(':')[1]+"'"+','
            text=text+"}"
        text=text+"},"
    text=text+"},"     
text=text.replace(',}}','}}')
text=text.replace('},},{','}},{')
text=text.replace("',}'","'}'")
print(text)


{'_key':1,'project_id':'85','project_name':'RA05-11_CMSRapeseed','experiment1':{'experiment_id':'133','experiment_name':'RfPPR','experiment_type':'wt vs mutant comparison','array_type':'CATMA','organism1':{'organ':'flower','organism_name':'Brassica napus','organism_id':'3'}}},{'_key':2,'project_id':'81','project_name':'CAGE-06_02_ddm1-VIBchips_PRIVATE','experiment1':{'experiment_id':'120','experiment_name':'seedlings_10j_Columbia_WT_vs_ddm1','experiment_type':'gene knock out','array_type':'CAGE','organism1':{'organ':'whole plant','organism_name':'Arabidopsis thaliana','organism_id':'1'}}},{'_key':3,'project_id':'184','project_name':'AFFY_seed_kinetic_Wheat','experiment1':{'experiment_id':'289','experiment_name':'AFFY_seed_kinetic_Wheat','experiment_type':'time course','array_type':'Affymetrix','organism1':{'organ':'seed','organism_name':'Triticum aestivum','organism_id':'4'}},},


In [23]:
#dictionlev2
text=''
cpt,cpt1=0,0
x=0
y=0
for key in diction.keys():
    cpt_special_col=0
    cpt=cpt+1

    for element in key.split('#'):
        cpt1=cpt1+1
        x=cpt1
        text=text+"'"+element.split(':')[0]+"':"+"['"+element.split(':')[1]+"','"+str(y)+'_'+str(x)+"'],"
    for el in diction[key]:
        for val in el.split('#'):
            text=text+"'"+val.split(':')[0]+"':'"+val.split(':')[1]+"'"+','
            #print(text)
        for el2 in dictionlev2[key+sepcol+el]:
            for val2 in el2.split('#'):
                text=text+"'"+val2.split(':')[0]+"':'"+val2.split(':')[1]+"'"+','
                cpt_special_col=cpt_special_col+1
    print(cpt_special_col)
    print(text)
    #print(text)

3
'project_id':['85','0_1'],'project_name':['RA05-11_CMSRapeseed','0_2'],'experiment_id':'133','experiment_name':'RfPPR','experiment_type':'wt vs mutant comparison','array_type':'CATMA','organ':'flower','organism_name':'Brassica napus','organism_id':'3',
3
'project_id':['85','0_1'],'project_name':['RA05-11_CMSRapeseed','0_2'],'experiment_id':'133','experiment_name':'RfPPR','experiment_type':'wt vs mutant comparison','array_type':'CATMA','organ':'flower','organism_name':'Brassica napus','organism_id':'3','project_id':['81','0_3'],'project_name':['CAGE-06_02_ddm1-VIBchips_PRIVATE','0_4'],'experiment_id':'120','experiment_name':'seedlings_10j_Columbia_WT_vs_ddm1','experiment_type':'gene knock out','array_type':'CAGE','organ':'whole plant','organism_name':'Arabidopsis thaliana','organism_id':'1',
3
'project_id':['85','0_1'],'project_name':['RA05-11_CMSRapeseed','0_2'],'experiment_id':'133','experiment_name':'RfPPR','experiment_type':'wt vs mutant comparison','array_type':'CATMA','organ':'f

444